In [141]:
import sqlite3
import numpy as np
import mysql.connector
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
plotly.tools.set_credentials_file(username='mainakchain', api_key='B9m4DVrYYmsoW3jDSiok')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
from scipy import stats
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import datetime
import holidays
from sklearn.model_selection import cross_val_score
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
import operator

In [192]:
time = pd.read_csv('time_between_two_tolls.csv')

In [194]:
time.toll_both.nunique()

10

In [187]:
toll_time = pd.read_csv('toll_time.csv')

In [188]:
toll_time = toll_time[toll_time['origin'] == 'Jamshedpur'][toll_time['destination'] == 'HOSUR']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [191]:
toll_time.toll_both.unique()

array(['Aganampudi', 'Attibele (BETL)', 'Bellupada', 'Bolapalli',
       'Budanam', 'Chilakapalem', 'Eethakota', 'Gaddurur',
       'Gudipada (Old Gangapada)', 'Gurapalli', 'Hasanpur (Sainkul)',
       'Hoskote', 'Kalaparru', 'Kaza', 'Krishnagiri', 'Krishnavaram',
       'Laxmipuram', 'Madapam', 'Main Toll (Gosthani Gate) (Sheela Nagar)',
       'Manguli', 'Mulbagal', 'Nathavalasa/ Vizianagaram', 'Palempalli',
       'Pallikonda', 'Panikholi', 'Pottipadu', 'Sergrarh',
       'Sunambatti (Musunur)', 'Tangutur', 'Unguturu', 'Vaniyambadi',
       'Vempadu', 'Venkatachallam &#160;(Old Nellore)'], dtype=object)

In [144]:
toll_time.drop(['Unnamed: 0'], axis=1, inplace=True)

In [145]:
new_toll_time = toll_time.copy()

In [146]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

In [147]:
new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

In [148]:
new_toll_time = toll_time.copy()

In [149]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

new_toll_time['unloading_in_time'] = new_toll_time['unloading_in_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['avg_travel_time'] = new_toll_time['start_date'] + (new_toll_time['unloading_in_time'] - new_toll_time['start_date'])/2

new_toll_time['month_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.month)

new_toll_time['ist_timestamp'] = new_toll_time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
new_toll_time['time_of_travel'] = new_toll_time.ist_timestamp.apply(lambda x: x.hour)

new_toll_time['day_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.weekday())

new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x: x.hour)

In [150]:
new_toll_time = new_toll_time[['toll_both','vehicle_no','loading_out_time','distance_travelled','time_of_travel','day_of_travel','month_of_travel','days_from_now','time_taken']]

In [151]:
toll_dict = {'Sergrarh':1,'Panikholi':2, 'Manguli':3, 'Gudipada (Old Gangapada)':4, 'Gurapalli':5, 'Bellupada':6, 'Madapam':7, 'Chilakapalem':8, 'Nathavalasa/ Vizianagaram':9, 'Laxmipuram':10, 'Vempadu':11, 'Krishnavaram':12, 'Kalaparru':13, 'Pottipadu':14, 'Kaza':15, 'Bolapalli':16, 'Sunambatti (Musunur)':17, 'Venkatachallam &#160;(Old Nellore)':18, 'Budanam':19, 'Gaddurur':20, 'Mulbagal':21, 'Hoskote':22, 'Attibele (BETL)':23, 'Aganampudi':24}

In [152]:
def find_toll_num(df):
    toll_dict = {}
    num = 1
    for toll in df.toll_both.unique().tolist():
        d = df['distance_travelled'][df.toll_both == toll].mean()
        toll_dict[toll] = d
        
    for key, value in sorted(toll_dict.iteritems(), key=lambda (k,v): (v,k)):
        new_toll_dict[key] = num
        num+=1
        
    return new_toll_dict

In [153]:
new_toll_dict = find_toll_num(new_toll_time)

In [154]:
train_df = new_toll_time.copy()
test_df= pd.DataFrame(columns=['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now'])
choice = True
while(choice):
    today_date = datetime.datetime.now()
    vehicle = raw_input('Enter the vehicle no.: ')
    start = raw_input('Enter the start date in %Y-%m-%d %H:%M:%S format: ')
    start_delta = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    
    loading_out = raw_input('Enter the loading out time in %Y-%m-%d %H:%M:%S format: ')
    loading_out_delta = datetime.datetime.strptime(loading_out, '%Y-%m-%d %H:%M:%S')
    
    a = []
    loop = input('Enter the number of tolls vehicle crossed sequentitially: ')
    count = 0
    print '      '
    while (count != loop):
        b = []
        toll = raw_input('Toll both: ')
        distance = input('Distance travelled from 1st toll in km: ')
        ist = raw_input('IST Timestamp in %Y-%m-%d %H:%M:%S format: ')
        ist_delta = datetime.datetime.strptime(ist, '%Y-%m-%d %H:%M:%S')
        b.extend([toll, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])
        a.append(b)
        count+=1
    
    add_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now', 'time_taken'])
    train_df = train_df.append(add_df)
    
    test_toll = raw_input('Enter the numbers of tolls the vehicle shall take in the journey sequentially: ')
    test_toll_list = test_toll.split(',')
    test_toll_list = [str(a) for a in test_toll_list]

    a = []

    for toll_val in test_toll_list:
        b = []
        distance = toll_time['distance_travelled'][toll_time.toll_both == toll_val].median()
        ist_delta = loading_out_delta + timedelta(hours=toll_time['time_taken'][toll_time.toll_both == toll_val].median())
        b.extend([toll_val, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days])

        a.append(b)
    add_test_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now'])
    
    test_df = test_df.append(add_test_df)
    
    c = raw_input('Want to enter more vehicles?(y/n)')
    
    if (c == 'y' or c == 'Y'):
        continue
    elif (c == 'n' or c == 'N'):
        choice = False
        print 'bbye'
    else:
        print 'Wrong Choice! Exiting'
        choice = False

Enter the vehicle no.: OR04M532M
Enter the start date in %Y-%m-%d %H:%M:%S format: 2017-11-02 00:00:00
Enter the loading out time in %Y-%m-%d %H:%M:%S format: 2017-11-03 12:02:43
Enter the number of tolls vehicle crossed sequentitially: 3
      
Toll both: Sergrarh
Distance travelled from 1st toll in km: 0
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 08:22:51
Toll both: Panikholi
Distance travelled from 1st toll in km: 95.1
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 10:42:12
Toll both: Manguli
Distance travelled from 1st toll in km: 146.8
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 12:10:12
Enter the numbers of tolls the vehicle shall take in the journey sequentially: Gudipada (Old Gangapada),Gurapalli,Bellupada,Madapam,Chilakapalem,Nathavalasa/ Vizianagaram,Laxmipuram,Vempadu,Krishnavaram,Kalaparru,Pottipadu,Kaza,Bolapalli,Sunambatti (Musunur),Venkatachallam &#160;(Old Nellore),Budanam,Gaddurur,Mulbagal,Hoskote,Attibele (BETL)
Want to enter more vehicles?

In [155]:
train_df.tail(10)

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken
4892,Venkatachallam &#160;(Old Nellore),WB23B7298,14,1165.77,9,4,12,15,90.697222
4893,Venkatachallam &#160;(Old Nellore),WB23C9411,23,1242.01,20,0,11,38,189.460833
0,Sergrarh,OR04M532M,12,0.00,8,5,11,40,20.335556
1,Panikholi,OR04M532M,12,95.10,10,5,11,40,22.658056
2,Manguli,OR04M532M,12,146.80,12,5,11,40,24.124722
0,Sergrarh,OR04M53MM,21,0.00,17,6,10,52,20.183611
1,Panikholi,OR04M53MM,21,94.60,19,6,10,52,22.575278
2,Manguli,OR04M53MM,21,146.50,21,6,10,52,24.169722
3,Gudipada (Old Gangapada),OR04M53MM,21,204.60,23,6,10,52,26.326389
4,Gurapalli,OR04M53MM,21,293.59,1,0,10,52,28.585278


In [156]:
test_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now
0,Gudipada (Old Gangapada),OR04M532M,12,203.320,10,6,11,40
1,Gurapalli,OR04M532M,12,291.250,13,6,11,40
2,Bellupada,OR04M532M,12,375.245,20,6,11,40
3,Madapam,OR04M532M,12,490.530,0,0,11,40
4,Chilakapalem,OR04M532M,12,517.415,3,0,11,40
5,Nathavalasa/ Vizianagaram,OR04M532M,12,556.620,5,0,11,40
6,Laxmipuram,OR04M532M,12,556.620,5,0,11,40
7,Vempadu,OR04M532M,12,684.270,11,0,11,40
8,Krishnavaram,OR04M532M,12,753.930,16,0,11,40
9,Kalaparru,OR04M532M,12,897.510,2,1,11,40


In [157]:
train_df.shape, test_df.shape

((998, 9), (38, 8))

In [158]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [96]:
# def change_toll_booth_to_dummy(df):
#     num = 1
#     dict_toll_both = {}
#     for toll in df['toll_both'].unique().tolist():
#         df['toll_both'] = df['toll_both'].replace({toll: num})
#         dict_toll_both[num] = toll
#         num += 1
#     return dict_toll_both

In [160]:
# dict_toll_both_train_df = change_toll_booth_to_dummy(train_df)
train_df.toll_both = train_df.toll_both.replace(new_toll_dict)
test_df.toll_both = test_df.toll_both.replace(new_toll_dict)

In [163]:
def change_vehicle_no_to_dummy(df):
    num = 1
    dict_vehicle_no = {}
    for truck in df['vehicle_no'].unique().tolist():
        df['vehicle_no'] = df['vehicle_no'].replace({truck: num})
        dict_vehicle_no[num] = truck
        num += 1
    return dict_vehicle_no

In [164]:
dict_vehicle_no_train_df = change_vehicle_no_to_dummy(train_df)

In [165]:
rev_dict_vehicle_no_test_df = dict((v,k) for k,v in dict_vehicle_no_train_df.iteritems())

In [166]:
test_df.vehicle_no = test_df.vehicle_no.replace(rev_dict_vehicle_no_test_df)

In [163]:
# MODEL BUILDING AND APPLICATION

In [167]:
train_X = train_df.drop(['time_taken'], axis=1)
train_y = train_df.time_taken

In [168]:
train_X.shape

(998, 8)

In [169]:
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.3, random_state=42)

In [170]:
reg_mlp = MLPRegressor(hidden_layer_sizes=(100,50), activation='logistic', alpha=8, solver='lbfgs', random_state=42)

In [171]:
scale= MinMaxScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

In [172]:
reg_mlp.fit(X_train_scaled, y_train)
reg_mlp.score(X_test_scaled, y_test) * 100

90.772662612156068

In [173]:
scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.86 (+/- 0.11)


In [174]:
scale = MinMaxScaler()
X_predict_scaled = scale.fit_transform(test_df)

In [175]:
predictions = reg_mlp.predict(X_predict_scaled)

In [176]:
show_df = test_df.copy()
show_df['predicted'] = predictions

In [177]:
show_df['predict_diff'] = show_df['predicted'].diff()

In [178]:
def med_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].median()

In [179]:
def mean_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].mean()

In [180]:
show_df['med_time'] = show_df['toll_both']
show_df['med_time'] = show_df.med_time.apply(lambda x:med_toll_time_taken(x))
show_df['mean_time'] = show_df['toll_both']
show_df['mean_time'] = show_df.mean_time.apply(lambda x:mean_toll_time_taken(x))

In [181]:
show_df['avg_pred_med'] = (show_df['predicted'] + show_df['med_time']) / 2

In [182]:
x = train_df
x['avg_speed'] = x.distance_travelled/ x.time_taken
x = x[x.vehicle_no == 25].sort_values('distance_travelled')
x['avg_speed'] = x.distance_travelled/(x.time_taken - 40.88)
x

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,avg_speed
793,2,25,23,0.00,16,3,11,24,40.885833,0.000000
704,3,25,23,90.70,21,3,11,24,45.974444,17.803708
576,4,25,23,147.17,5,3,11,24,53.906389,11.297836
235,5,25,23,204.06,6,3,11,24,55.247222,14.203163
280,6,25,23,292.31,9,3,11,24,58.063611,17.010976
27,7,25,23,377.35,13,3,11,24,61.943889,17.914546
528,8,25,23,493.02,17,3,11,24,66.076667,19.566874
154,9,25,23,520.71,18,3,11,24,66.876667,20.029876
657,11,25,23,560.93,19,3,11,24,67.979444,20.698948
484,10,25,23,560.93,19,3,11,24,67.979444,20.698948


In [183]:
train_df[train_df['vehicle_no'] == 45]

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,avg_speed
990,2,45,12,0.0,8,5,11,40,20.335556,0.000000
991,3,45,12,95.1,10,5,11,40,22.658056,4.197183
992,4,45,12,146.8,12,5,11,40,24.124722,6.085044


In [184]:
show_df['avg_speed_pred'] = show_df['distance_travelled']/show_df['predicted']
show_df['avg_speed_med'] = show_df['distance_travelled']/show_df['med_time']

In [185]:
show_df['test_pred'] = show_df['predicted'] + show_df.avg_speed_pred

In [186]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,predict_diff,med_time,mean_time,avg_pred_med,avg_speed_pred,avg_speed_med,test_pred
0,5,45,12,203.320,10,6,11,40,33.800109,NaN,45.996250,50.308219,39.898179,6.015365,4.420360,39.815474
1,6,45,12,291.250,13,6,11,40,37.813344,4.013235,49.686667,52.731903,43.750005,7.702307,5.861734,45.515651
2,7,45,12,375.245,20,6,11,40,40.685817,2.872473,56.755556,61.037500,48.720686,9.222993,6.611599,49.908810
3,8,45,12,490.530,0,0,11,40,83.496659,42.810842,60.303056,65.169290,71.899857,5.874846,8.134414,89.371505
4,9,45,12,517.415,3,0,11,40,85.026134,1.529474,63.638889,66.576692,74.332511,6.085364,8.130485,91.111498
5,11,45,12,556.620,5,0,11,40,89.201325,4.175191,65.382917,68.902639,77.292121,6.240042,8.513233,95.441367
6,10,45,12,556.620,5,0,11,40,87.441366,-1.759958,65.382917,68.901881,76.412142,6.365637,8.513233,93.807003
7,14,45,12,684.270,11,0,11,40,97.031446,9.590079,71.403333,75.455189,84.217390,7.052044,9.583166,104.083490
8,15,45,12,753.930,16,0,11,40,97.896329,0.864883,76.884444,80.255975,87.390387,7.701310,9.806015,105.597639
9,18,45,12,897.510,2,1,11,40,84.731795,-13.164534,86.089444,92.056926,85.410620,10.592364,10.425320,95.324159
